In [1]:
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.neighbors import NearestNeighbors


In [2]:
def rfm_variables(df):
  ## quantidade de vendas
  f_sales = len(df)
  ## quantidade de vendas com lucro
  f_profit = len(df[df['Profit'] > 0])
  ## total de vendas, lucro, quantidade e custo de entrega
  m_sales = round(df['Sales'].sum(),2)
  m_profit = round(df['Profit'].sum(),2)
  m_qty = df['Quantity'].sum()
  ## periodicidade
  df_sort = df[['Order Date']].sort_values(by='Order Date').drop_duplicates()
  df_sort['diff'] = df_sort['Order Date'] - df_sort['Order Date'].shift(1)
  df_sort['diff_int'] = df_sort['diff'].dt.days
  r_days = round(df_sort['diff_int'].mean(),2)
  return f_sales, f_profit, m_sales, m_profit, m_qty, r_days

In [3]:
def fit_data(data, variable):
  rfm = pd.DataFrame()
  variaveis = data[variable].unique()
  for variavel in variaveis:
    dados = data[data[variable] == variavel]
    f_vendas, f_lucro, m_vendas, m_lucro, m_qtde, r_dias = rfm_variables(dados)
    rfm = rfm.append(
        {
          'referencia': variavel, 
          'm_vendas': m_vendas, 
          'm_lucro': m_lucro,
          'm_qtde': m_qtde, 
          'r_dias': r_dias,
          'f_vendas': f_vendas, 
          'f_lucro': f_lucro           
        },
        ignore_index = True
    )
  return rfm

In [4]:
data = pd.read_feather('tavbase/Sample.feather')


In [5]:
variaveis = [
    'f_vendas',
    'f_lucro',
    'm_lucro',
    'm_qtde', 
    'm_vendas',
    'r_dias'
]

In [6]:
## Aula 09 - Clusterização ou Agrupamento
state_rfm = fit_data(data, 'State')
state_rfm = state_rfm.fillna(0)
state_rfm['cluster'] = KMeans(
    n_clusters=3,
    random_state=0
).fit(
    state_rfm[
        variaveis
    ]
).labels_
state_rfm

C:\Users\duduf\AppData\Local\Temp\ipykernel_25576\2601909467.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rfm = rfm.append(
C:\Users\duduf\AppData\Local\Temp\ipykernel_25576\2601909467.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rfm = rfm.append(
C:\Users\duduf\AppData\Local\Temp\ipykernel_25576\2601909467.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rfm = rfm.append(
C:\Users\duduf\AppData\Local\Temp\ipykernel_25576\2601909467.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rfm = rfm.append(
C:\Users\duduf\AppData\Local\Temp\ipykernel_25576\2601909467.py:7: FutureWarning: The frame.append method is deprecated 

,referencia,m_vendas,m_lucro,m_qtde,r_dias,f_vendas,f_lucro,cluster
0,Kentucky,36591.75,11199.70,523,24.63,139,138,0
1,California,457687.63,76381.39,7667,2.21,2001,1884,1
2,Florida,89473.71,-3399.30,1379,7.75,383,252,2
3,North Carolina,55603.16,-7490.91,983,10.66,249,180,0
4,Washington,138641.27,33402.65,1883,6.54,506,483,2
5,Texas,170188.05,-25729.36,3724,3.73,985,493,2
6,Wisconsin,32114.61,8401.80,463,27.23,110,110,0
7,Utah,11220.06,2546.53,219,53.16,53,53,0
8,Nebraska,7464.93,2037.09,136,57.00,38,38,0
9,Pennsylvania,116511.91,-15559.96,2153,5.70,587,303,2


In [7]:
cluster = pd.DataFrame()
for index, row in enumerate(KMeans(n_clusters=3, random_state=0
    ).fit(state_rfm[variaveis]).cluster_centers_):
    print(row)
    cluster = cluster.append(
        {
            'cluster'    : index,
            'clf_vendas' : row[0],
            'cls_lucro'  : row[1],
            'clm_lucro'  : row[2],
            'clm_qtde'   : row[3],
            'clm_vendas' : row[4],
            'clr_dias'   : row[5]
        },
        ignore_index = True
    )
cluster  

[   76.            66.69230769  3430.28717949   292.30769231
 18166.44717949    77.67564103]
[1.5645000e+03 1.4790000e+03 7.5209970e+04 5.9455000e+03 3.8428195e+05
 2.7800000e+00]
[4.87625000e+02 3.12375000e+02 2.74487500e+02 1.82275000e+03
 1.02518189e+05 7.89875000e+00]


C:\Users\duduf\AppData\Local\Temp\ipykernel_25576\4237266428.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cluster = cluster.append(
C:\Users\duduf\AppData\Local\Temp\ipykernel_25576\4237266428.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cluster = cluster.append(
C:\Users\duduf\AppData\Local\Temp\ipykernel_25576\4237266428.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cluster = cluster.append(


,cluster,clf_vendas,cls_lucro,clm_lucro,clm_qtde,clm_vendas,clr_dias
0,0.0,76.000,66.692308,3430.287179,292.307692,18166.447179,77.675641
1,1.0,1564.500,1479.000000,75209.970000,5945.500000,384281.950000,2.780000
2,2.0,487.625,312.375000,274.487500,1822.750000,102518.188750,7.898750


In [8]:
state_rfm = state_rfm.merge(
    cluster,
    on='cluster',
    how='left'
)
state_rfm

,referencia,m_vendas,m_lucro,m_qtde,r_dias,f_vendas,f_lucro,cluster,clf_vendas,cls_lucro,clm_lucro,clm_qtde,clm_vendas,clr_dias
0,Kentucky,36591.75,11199.70,523,24.63,139,138,0,76.000,66.692308,3430.287179,292.307692,18166.447179,77.675641
1,California,457687.63,76381.39,7667,2.21,2001,1884,1,1564.500,1479.000000,75209.970000,5945.500000,384281.950000,2.780000
2,Florida,89473.71,-3399.30,1379,7.75,383,252,2,487.625,312.375000,274.487500,1822.750000,102518.188750,7.898750
3,North Carolina,55603.16,-7490.91,983,10.66,249,180,0,76.000,66.692308,3430.287179,292.307692,18166.447179,77.675641
4,Washington,138641.27,33402.65,1883,6.54,506,483,2,487.625,312.375000,274.487500,1822.750000,102518.188750,7.898750
5,Texas,170188.05,-25729.36,3724,3.73,985,493,2,487.625,312.375000,274.487500,1822.750000,102518.188750,7.898750
6,Wisconsin,32114.61,8401.80,463,27.23,110,110,0,76.000,66.692308,3430.287179,292.307692,18166.447179,77.675641
7,Utah,11220.06,2546.53,219,53.16,53,53,0,76.000,66.692308,3430.287179,292.307692,18166.447179,77.675641
8,Nebraska,7464.93,2037.09,136,57.00,38,38,0,76.000,66.692308,3430.287179,292.307692,18166.447179,77.675641
9,Pennsylvania,116511.91,-15559.96,2153,5.70,587,303,2,487.625,312.375000,274.487500,1822.750000,102518.188750,7.898750


In [9]:
state_rfm.to_feather('tavbase/clusterizacao_estado.feather')

variaveis = [
    'f_vendas',
    'f_lucro',
    'm_lucro',
    'm_qtde', 
    'm_vendas',
    'r_dias'
]
original = fit_data(data, 'State')
original = original.fillna(0)
base = original[variaveis]

C:\Users\duduf\AppData\Local\Temp\ipykernel_25576\2601909467.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rfm = rfm.append(
C:\Users\duduf\AppData\Local\Temp\ipykernel_25576\2601909467.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rfm = rfm.append(
C:\Users\duduf\AppData\Local\Temp\ipykernel_25576\2601909467.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rfm = rfm.append(
C:\Users\duduf\AppData\Local\Temp\ipykernel_25576\2601909467.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rfm = rfm.append(
C:\Users\duduf\AppData\Local\Temp\ipykernel_25576\2601909467.py:7: FutureWarning: The frame.append method is deprecated 

In [10]:
vizinhos = NearestNeighbors(n_neighbors=min(4, len(base))).fit(base)
similares = pd.DataFrame()
for index, row in original.iterrows():
    print('Referencia: {0}'.format(row['referencia']))
    print('Referencias Similares:')
    original_referencia = original[original['referencia'] == row['referencia']][variaveis]
    similar = vizinhos.kneighbors(original_referencia, return_distance=False)[0]
    original_similar = original.iloc[similar][variaveis].reset_index()
    referencia = original.iloc[similar]['referencia'].reset_index()
    referencia = referencia.merge(original_similar, on='index', how='left')
    referencia = referencia.drop(columns=['index'])
    for ind, rw in referencia.iterrows():    
        if row['referencia'] != rw['referencia']:            
            print('--> {0}'.format(rw['referencia']))
            similares = similares.append({'referencia': row['referencia'], 'vizinho': rw['referencia']}, ignore_index = True)
similares

C:\Users\duduf\AppData\Local\Temp\ipykernel_25576\4098855849.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  similares = similares.append({'referencia': row['referencia'], 'vizinho': rw['referencia']}, ignore_index = True)


Referencia: Kentucky
Referencias Similares:
--> New Jersey
--> Wisconsin
--> Minnesota
Referencia: California
Referencias Similares:
--> New York
--> Texas
--> Washington
Referencia: Florida
Referencias Similares:
--> Illinois
--> Ohio
--> Virginia
Referencia: North Carolina
Referencias Similares:
--> Arizona
--> Colorado
--> Ohio
Referencia: Washington
Referencias Similares:
--> Pennsylvania
--> Florida
--> Michigan
Referencia: Texas
Referencias Similares:
--> Pennsylvania
--> Washington
--> Florida
Referencia: Wisconsin
Referencias Similares:
--> Minnesota
--> Massachusetts
--> New Jersey
Referencia: Utah
Referencias Similares:
--> Mississippi
--> Arkansas
--> Louisiana
Referencia: Nebraska
Referencias Similares:


C:\Users\duduf\AppData\Local\Temp\ipykernel_25576\4098855849.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  similares = similares.append({'referencia': row['referencia'], 'vizinho': rw['referencia']}, ignore_index = True)
C:\Users\duduf\AppData\Local\Temp\ipykernel_25576\4098855849.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  similares = similares.append({'referencia': row['referencia'], 'vizinho': rw['referencia']}, ignore_index = True)
C:\Users\duduf\AppData\Local\Temp\ipykernel_25576\4098855849.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  similares = similares.append({'referencia': row['referencia'], 'vizinho': rw['referencia']}, ignore_index = True)
C:\Users\duduf\AppData\Local\Temp\ipykernel_25576

--> New Hampshire
--> South Carolina
--> Vermont
Referencia: Pennsylvania
Referencias Similares:
--> Florida
--> Illinois
--> Ohio
Referencia: Illinois
Referencias Similares:
--> Ohio
--> Florida
--> North Carolina
Referencia: Minnesota
Referencias Similares:
--> Delaware
--> Wisconsin
--> Massachusetts
Referencia: Michigan
Referencias Similares:
--> Virginia
--> Indiana
--> Georgia
Referencia: Delaware
Referencias Similares:
--> Minnesota
--> Massachusetts
--> Maryland
Referencia: Indiana
Referencias Similares:
--> Georgia
--> Virginia
--> Kentucky
Referencia: New York
Referencias Similares:
--> California
--> Texas
--> Washington
Referencia: Arizona
Referencias Similares:
--> Colorado


C:\Users\duduf\AppData\Local\Temp\ipykernel_25576\4098855849.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  similares = similares.append({'referencia': row['referencia'], 'vizinho': rw['referencia']}, ignore_index = True)
C:\Users\duduf\AppData\Local\Temp\ipykernel_25576\4098855849.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  similares = similares.append({'referencia': row['referencia'], 'vizinho': rw['referencia']}, ignore_index = True)
C:\Users\duduf\AppData\Local\Temp\ipykernel_25576\4098855849.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  similares = similares.append({'referencia': row['referencia'], 'vizinho': rw['referencia']}, ignore_index = True)
C:\Users\duduf\AppData\Local\Temp\ipykernel_25576

--> Tennessee
--> Massachusetts
Referencia: Virginia
Referencias Similares:
--> Michigan
--> Indiana
--> Georgia
Referencia: Tennessee
Referencias Similares:
--> Colorado
--> Arizona
--> Massachusetts
Referencia: Alabama
Referencias Similares:
--> Oklahoma
--> Missouri
--> Rhode Island
Referencia: South Carolina
Referencias Similares:
--> Vermont
--> Louisiana
--> Nebraska
Referencia: Oregon
Referencias Similares:
--> Nevada
--> Connecticut
--> Oklahoma
Referencia: Colorado
Referencias Similares:
--> Tennessee
--> Arizona
--> Massachusetts
Referencia: Iowa
Referencias Similares:
--> New Mexico
--> Idaho
--> Montana
Referencia: Ohio
Referencias Similares:
--> Illinois
--> Florida
--> North Carolina
Referencia: Missouri
Referencias Similares:
--> Rhode Island
--> Maryland
--> Alabama
Referencia: Oklahoma
Referencias Similares:
--> Alabama
--> Missouri
--> Nevada
Referencia: New Mexico
Referencias Similares:
--> Iowa
--> Idaho
--> Montana
Referencia: Louisiana
Referencias Similares:
--> V

C:\Users\duduf\AppData\Local\Temp\ipykernel_25576\4098855849.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  similares = similares.append({'referencia': row['referencia'], 'vizinho': rw['referencia']}, ignore_index = True)
C:\Users\duduf\AppData\Local\Temp\ipykernel_25576\4098855849.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  similares = similares.append({'referencia': row['referencia'], 'vizinho': rw['referencia']}, ignore_index = True)
C:\Users\duduf\AppData\Local\Temp\ipykernel_25576\4098855849.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  similares = similares.append({'referencia': row['referencia'], 'vizinho': rw['referencia']}, ignore_index = True)
C:\Users\duduf\AppData\Local\Temp\ipykernel_25576

--> Alabama
Referencia: Rhode Island
Referencias Similares:
--> Missouri
--> Maryland
--> Alabama
Referencia: Mississippi
Referencias Similares:
--> Utah
--> Arkansas
--> Louisiana
Referencia: Arkansas
Referencias Similares:
--> Mississippi
--> Utah
--> Connecticut
Referencia: Montana
Referencias Similares:
--> New Mexico
--> Iowa
--> Idaho
Referencia: New Hampshire
Referencias Similares:
--> Nebraska
--> South Carolina
--> Montana
Referencia: Maryland
Referencias Similares:
--> Rhode Island
--> Missouri
--> Alabama
Referencia: District of Columbia
Referencias Similares:
--> Kansas
--> Idaho
--> Wyoming
Referencia: Kansas
Referencias Similares:
--> District of Columbia
--> Idaho
--> Wyoming
Referencia: Vermont
Referencias Similares:
--> Louisiana
--> South Carolina
--> Nebraska
Referencia: Maine
Referencias Similares:
--> South Dakota
--> West Virginia
--> North Dakota
Referencia: South Dakota
Referencias Similares:
--> Maine
--> West Virginia
--> North Dakota
Referencia: Idaho
Referen

C:\Users\duduf\AppData\Local\Temp\ipykernel_25576\4098855849.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  similares = similares.append({'referencia': row['referencia'], 'vizinho': rw['referencia']}, ignore_index = True)
C:\Users\duduf\AppData\Local\Temp\ipykernel_25576\4098855849.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  similares = similares.append({'referencia': row['referencia'], 'vizinho': rw['referencia']}, ignore_index = True)
C:\Users\duduf\AppData\Local\Temp\ipykernel_25576\4098855849.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  similares = similares.append({'referencia': row['referencia'], 'vizinho': rw['referencia']}, ignore_index = True)
C:\Users\duduf\AppData\Local\Temp\ipykernel_25576

,referencia,vizinho
0,Kentucky,New Jersey
1,Kentucky,Wisconsin
2,Kentucky,Minnesota
3,California,New York
4,California,Texas
...,...,...
142,Wyoming,South Dakota
143,Wyoming,Maine
144,West Virginia,North Dakota
145,West Virginia,South Dakota


In [11]:
similares.to_feather('tavbase/knn_estado.feather')

In [12]:
## Aula 15 - KNN - Vizinhos mais próximos
original = fit_data(data, 'Category')
original = original.fillna(0)
base = original[variaveis]
vizinhos = NearestNeighbors(n_neighbors=min(4, len(base))).fit(base)
similares = pd.DataFrame()
for index, row in original.iterrows():
    print('Referencia: {0}'.format(row['referencia']))
    print('Referencias Similares:')
    original_referencia = original[original['referencia'] == row['referencia']][variaveis]
    similar = vizinhos.kneighbors(original_referencia, return_distance=False)[0]
    original_similar = original.iloc[similar][variaveis].reset_index()
    referencia = original.iloc[similar]['referencia'].reset_index()
    referencia = referencia.merge(original_similar, on='index', how='left')
    referencia = referencia.drop(columns=['index'])
    for ind, rw in referencia.iterrows():    
        if row['referencia'] != rw['referencia']:            
            print('--> {0}'.format(rw['referencia']))
            similares = similares.append({'referencia': row['referencia'], 'vizinho': rw['referencia']}, ignore_index = True)
similares

similares.to_feather('tavbase/knn_categoria.feather')

Referencia: Furniture
Referencias Similares:
--> Office Supplies
--> Technology
Referencia: Office Supplies
Referencias Similares:
--> Furniture
--> Technology
Referencia: Technology
Referencias Similares:
--> Office Supplies
--> Furniture


C:\Users\duduf\AppData\Local\Temp\ipykernel_25576\2601909467.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rfm = rfm.append(
C:\Users\duduf\AppData\Local\Temp\ipykernel_25576\2601909467.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rfm = rfm.append(
C:\Users\duduf\AppData\Local\Temp\ipykernel_25576\2601909467.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rfm = rfm.append(
C:\Users\duduf\AppData\Local\Temp\ipykernel_25576\3973162273.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  similares = similares.append({'referencia': row['referencia'], 'vizinho': rw['referencia']}, ignore_index = True)
C:\Users\duduf\AppData\

In [13]:
## Aula 15 - KNN - Vizinhos mais próximos
original = fit_data(data, 'Sub-Category')
original = original.fillna(0)
base = original[variaveis]
vizinhos = NearestNeighbors(n_neighbors=min(4, len(base))).fit(base)
similares = pd.DataFrame()
for index, row in original.iterrows():
    print('Referencia: {0}'.format(row['referencia']))
    print('Referencias Similares:')
    original_referencia = original[original['referencia'] == row['referencia']][variaveis]
    similar = vizinhos.kneighbors(original_referencia, return_distance=False)[0]
    original_similar = original.iloc[similar][variaveis].reset_index()
    referencia = original.iloc[similar]['referencia'].reset_index()
    referencia = referencia.merge(original_similar, on='index', how='left')
    referencia = referencia.drop(columns=['index'])
    for ind, rw in referencia.iterrows():    
        if row['referencia'] != rw['referencia']:            
            print('--> {0}'.format(rw['referencia']))
            similares = similares.append({'referencia': row['referencia'], 'vizinho': rw['referencia']}, ignore_index = True)
similares

similares.to_feather('tavbase/knn_subcategoria.feather')

C:\Users\duduf\AppData\Local\Temp\ipykernel_25576\2601909467.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rfm = rfm.append(
C:\Users\duduf\AppData\Local\Temp\ipykernel_25576\2601909467.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rfm = rfm.append(
C:\Users\duduf\AppData\Local\Temp\ipykernel_25576\2601909467.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rfm = rfm.append(
C:\Users\duduf\AppData\Local\Temp\ipykernel_25576\2601909467.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rfm = rfm.append(
C:\Users\duduf\AppData\Local\Temp\ipykernel_25576\2601909467.py:7: FutureWarning: The frame.append method is deprecated 

Referencia: Bookcases
Referencias Similares:
--> Appliances
--> Furnishings
--> Paper
Referencia: Chairs
Referencias Similares:
--> Phones
--> Storage
--> Binders
Referencia: Labels
Referencias Similares:
--> Envelopes
--> Fasteners
--> Art
Referencia: Tables
Referencias Similares:
--> Machines
--> Storage
--> Binders
Referencia: Storage
Referencias Similares:
--> Binders
--> Machines
--> Tables
Referencia: Furnishings
Referencias Similares:
--> Appliances
--> Paper
--> Bookcases
Referencia: Art
Referencias Similares:
--> Envelopes
--> Labels
--> Supplies
Referencia: Phones
Referencias Similares:
--> Chairs
--> Storage
--> Binders
Referencia: Binders
Referencias Similares:
--> Storage
--> Machines
--> Accessories


C:\Users\duduf\AppData\Local\Temp\ipykernel_25576\1717799548.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  similares = similares.append({'referencia': row['referencia'], 'vizinho': rw['referencia']}, ignore_index = True)
C:\Users\duduf\AppData\Local\Temp\ipykernel_25576\1717799548.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  similares = similares.append({'referencia': row['referencia'], 'vizinho': rw['referencia']}, ignore_index = True)
C:\Users\duduf\AppData\Local\Temp\ipykernel_25576\1717799548.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  similares = similares.append({'referencia': row['referencia'], 'vizinho': rw['referencia']}, ignore_index = True)
C:\Users\duduf\AppData\Local\Temp\ipykernel_25576

Referencia: Appliances
Referencias Similares:
--> Furnishings
--> Bookcases
--> Paper
Referencia: Paper
Referencias Similares:
--> Furnishings
--> Appliances
--> Supplies
Referencia: Accessories
Referencias Similares:
--> Copiers
--> Binders
--> Machines
Referencia: Envelopes
Referencias Similares:
--> Labels
--> Art
--> Fasteners
Referencia: Fasteners
Referencias Similares:
--> Labels
--> Envelopes
--> Art
Referencia: Supplies
Referencias Similares:
--> Art
--> Envelopes
--> Labels
Referencia: Machines
Referencias Similares:
--> Tables
--> Binders
--> Storage
Referencia: Copiers
Referencias Similares:
--> Accessories
--> Appliances
--> Binders


C:\Users\duduf\AppData\Local\Temp\ipykernel_25576\1717799548.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  similares = similares.append({'referencia': row['referencia'], 'vizinho': rw['referencia']}, ignore_index = True)
C:\Users\duduf\AppData\Local\Temp\ipykernel_25576\1717799548.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  similares = similares.append({'referencia': row['referencia'], 'vizinho': rw['referencia']}, ignore_index = True)
C:\Users\duduf\AppData\Local\Temp\ipykernel_25576\1717799548.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  similares = similares.append({'referencia': row['referencia'], 'vizinho': rw['referencia']}, ignore_index = True)
C:\Users\duduf\AppData\Local\Temp\ipykernel_25576

In [14]:
## Aula 15 - KNN - Vizinhos mais próximos
original = fit_data(data, 'Product Name')
original = original.fillna(0)
base = original[variaveis]
vizinhos = NearestNeighbors(n_neighbors=min(4, len(base))).fit(base)
similares = pd.DataFrame()
for index, row in original.iterrows():
    print('Referencia: {0}'.format(row['referencia']))
    print('Referencias Similares:')
    original_referencia = original[original['referencia'] == row['referencia']][variaveis]
    similar = vizinhos.kneighbors(original_referencia, return_distance=False)[0]
    original_similar = original.iloc[similar][variaveis].reset_index()
    referencia = original.iloc[similar]['referencia'].reset_index()
    referencia = referencia.merge(original_similar, on='index', how='left')
    referencia = referencia.drop(columns=['index'])
    for ind, rw in referencia.iterrows():    
        if row['referencia'] != rw['referencia']:            
            print('--> {0}'.format(rw['referencia']))
            similares = similares.append({'referencia': row['referencia'], 'vizinho': rw['referencia']}, ignore_index = True)
similares

similares.to_feather('tavbase/knn_produto.feather')

C:\Users\duduf\AppData\Local\Temp\ipykernel_25576\2601909467.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rfm = rfm.append(
C:\Users\duduf\AppData\Local\Temp\ipykernel_25576\2601909467.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rfm = rfm.append(
C:\Users\duduf\AppData\Local\Temp\ipykernel_25576\2601909467.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rfm = rfm.append(
C:\Users\duduf\AppData\Local\Temp\ipykernel_25576\2601909467.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rfm = rfm.append(
C:\Users\duduf\AppData\Local\Temp\ipykernel_25576\2601909467.py:7: FutureWarning: The frame.append method is deprecated 

Referencia: Bush Somerset Collection Bookcase
Referencias Similares:
--> Office Star - Ergonomically Designed Knee Chair
--> Case Logic 2.4GHz Wireless Keyboard
--> Tennsco Industrial Shelving
Referencia: Hon Deluxe Fabric Upholstered Stacking Chairs, Rounded Back
Referencias Similares:
--> Tennsco 6- and 18-Compartment Lockers
--> Logitech P710e Mobile Speakerphone
--> Plantronics CS510 - Over-the-Head monaural Wireless Headset System
Referencia: Self-Adhesive Address Labels for Typewriters by Universal
Referencias Similares:
--> Xerox 1993
--> #10- 4 1/8" x 9 1/2" Security-Tint Envelopes
--> Xerox 217
Referencia: Bretford CR4500 Series Slim Rectangular Table
Referencias Similares:
--> Hon 94000 Series Round Tables
--> Bevis Oval Conference Table, Walnut
--> Samsung Galaxy S III - 16GB - pebble blue (T-Mobile)
Referencia: Eldon Fold 'N Roll Cart System
Referencias Similares:
--> Boston Home & Office Model 2000 Electric Pencil Sharpeners
--> Fellowes Personal Hanging Folder Files, Navy

C:\Users\duduf\AppData\Local\Temp\ipykernel_25576\3162557540.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  similares = similares.append({'referencia': row['referencia'], 'vizinho': rw['referencia']}, ignore_index = True)
C:\Users\duduf\AppData\Local\Temp\ipykernel_25576\3162557540.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  similares = similares.append({'referencia': row['referencia'], 'vizinho': rw['referencia']}, ignore_index = True)
C:\Users\duduf\AppData\Local\Temp\ipykernel_25576\3162557540.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  similares = similares.append({'referencia': row['referencia'], 'vizinho': rw['referencia']}, ignore_index = True)
C:\Users\duduf\AppData\Local\Temp\ipykernel_25576

--> Seth Thomas 8 1/2" Cubicle Clock
Referencia: Belkin F5C206VTEL 6 Outlet Surge
Referencias Similares:
--> Personal Filing Tote with Lid, Black/Gray
--> Poly Designer Cover & Back
--> Newell 345
Referencia: Chromcraft Rectangular Conference Tables
Referencias Similares:
--> Chromcraft 48" x 96" Racetrack Double Pedestal Table
--> SAFCO Boltless Steel Shelving
--> High-Back Leather Manager's Chair
Referencia: Konftel 250 Conference phone - Charcoal black
Referencias Similares:
--> Plantronics HL10 Handset Lifter
--> Jabra Supreme Plus Driver Edition Headset
--> Mitel 5320 IP Phone VoIP phone
Referencia: Xerox 1967
Referencias Similares:
--> Xerox 200
--> Avery 483
--> Xerox 219
Referencia: Fellowes PB200 Plastic Comb Binding Machine
Referencias Similares:
--> Wireless Extenders zBoost YX545 SOHO Signal Booster
--> Hewlett Packard 310 Color Digital Copier
--> Cisco SPA301
Referencia: Holmes Replacement Filter for HEPA Air Cleaner, Very Large Room, HEPA Filter
Referencias Similares:
-->

C:\Users\duduf\AppData\Local\Temp\ipykernel_25576\3162557540.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  similares = similares.append({'referencia': row['referencia'], 'vizinho': rw['referencia']}, ignore_index = True)
C:\Users\duduf\AppData\Local\Temp\ipykernel_25576\3162557540.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  similares = similares.append({'referencia': row['referencia'], 'vizinho': rw['referencia']}, ignore_index = True)
C:\Users\duduf\AppData\Local\Temp\ipykernel_25576\3162557540.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  similares = similares.append({'referencia': row['referencia'], 'vizinho': rw['referencia']}, ignore_index = True)
C:\Users\duduf\AppData\Local\Temp\ipykernel_25576

...

C:\Users\duduf\AppData\Local\Temp\ipykernel_25576\3162557540.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  similares = similares.append({'referencia': row['referencia'], 'vizinho': rw['referencia']}, ignore_index = True)
C:\Users\duduf\AppData\Local\Temp\ipykernel_25576\3162557540.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  similares = similares.append({'referencia': row['referencia'], 'vizinho': rw['referencia']}, ignore_index = True)
C:\Users\duduf\AppData\Local\Temp\ipykernel_25576\3162557540.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  similares = similares.append({'referencia': row['referencia'], 'vizinho': rw['referencia']}, ignore_index = True)
C:\Users\duduf\AppData\Local\Temp\ipykernel_25576

--> Eldon File Chest Portable File
Referencia: Lexmark 20R1285 X6650 Wireless All-in-One Printer
Referencias Similares:
--> Microsoft Wireless Mobile Mouse 4000
--> Xerox 1911
--> Tuf-Vin Binders
Referencia: Nokia Lumia 1020
Referencias Similares:
--> Hewlett-Packard Deskjet 6540 Color Inkjet Printer
--> Bush Westfield Collection Bookcases, Fully Assembled
--> Global Leather Task Chair, Black
Referencia: Acme Serrated Blade Letter Opener
Referencias Similares:
--> Avery 479
--> Alliance Rubber Bands
--> Hammermill CopyPlus Copy Paper (20Lb. and 84 Bright)
Referencia: Okidata B401 Printer
Referencias Similares:
--> Epson Perfection V600 Photo Scanner
--> Cisco 8961 IP Phone Charcoal
--> Bush Westfield Collection Bookcases, Dark Cherry Finish, Fully Assembled
Referencia: Xerox Blank Computer Paper
Referencias Similares:
--> Linden 12" Wall Clock With Oak Frame
--> Snap-A-Way Black Print Carbonless Speed Message, No Reply Area, Duplicate
--> Xerox 1932
Referencia: 4009 Highlighters
Refere

C:\Users\duduf\AppData\Local\Temp\ipykernel_25576\3162557540.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  similares = similares.append({'referencia': row['referencia'], 'vizinho': rw['referencia']}, ignore_index = True)
C:\Users\duduf\AppData\Local\Temp\ipykernel_25576\3162557540.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  similares = similares.append({'referencia': row['referencia'], 'vizinho': rw['referencia']}, ignore_index = True)
C:\Users\duduf\AppData\Local\Temp\ipykernel_25576\3162557540.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  similares = similares.append({'referencia': row['referencia'], 'vizinho': rw['referencia']}, ignore_index = True)
C:\Users\duduf\AppData\Local\Temp\ipykernel_25576

--> Dixon Ticonderoga Core-Lock Colored Pencils, 48-Color Set
Referencia: Grip Seal Envelopes
Referencias Similares:
--> Xerox 1989
--> Avery Hi-Liter Pen Style Six-Color Fluorescent Set
--> Xerox 20
Referencia: Penpower WorldCard Pro Card Scanner
Referencias Similares:
--> Eldon File Chest Portable File
--> Ultra Commercial Grade Dual Valve Door Closer
--> Bush Saratoga Collection 5-Shelf Bookcase, Hanover Cherry, *Special Order
Referencia: Sony 8GB Class 10 Micro SDHC R40 Memory Card
Referencias Similares:
--> SanDisk Cruzer 8 GB USB Flash Drive
--> Sterling Rubber Bands by Alliance
--> Eldon Jumbo ProFile Portable File Boxes Graphite/Black
Referencia: RCA ViSYS 25425RE1 Corded phone
Referencias Similares:
--> Wasp CCD Handheld Bar Code Reader
--> Lexmark S315 Color Inkjet Printer
--> Microsoft Arc Touch Mouse
Referencia: Cisco 8961 IP Phone Charcoal
Referencias Similares:
--> Epson Perfection V600 Photo Scanner
--> Honeywell Enviracaire Portable Air Cleaner for up to 8 x 10 Room
-->

C:\Users\duduf\AppData\Local\Temp\ipykernel_25576\3162557540.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  similares = similares.append({'referencia': row['referencia'], 'vizinho': rw['referencia']}, ignore_index = True)
C:\Users\duduf\AppData\Local\Temp\ipykernel_25576\3162557540.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  similares = similares.append({'referencia': row['referencia'], 'vizinho': rw['referencia']}, ignore_index = True)
C:\Users\duduf\AppData\Local\Temp\ipykernel_25576\3162557540.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  similares = similares.append({'referencia': row['referencia'], 'vizinho': rw['referencia']}, ignore_index = True)
C:\Users\duduf\AppData\Local\Temp\ipykernel_25576

In [15]:
original = fit_data(data, 'State')
original = original.fillna(0)
base = original[variaveis]
vizinhos = NearestNeighbors(n_neighbors=min(4, len(base))).fit(base)
similares = pd.DataFrame()
for index, row in original.iterrows():
    print('Referencia: {0}'.format(row['referencia']))
    print('Referencias Similares:')
    original_referencia = original[original['referencia'] == row['referencia']][variaveis]
    similar = vizinhos.kneighbors(original_referencia, return_distance=False)[0]
    original_similar = original.iloc[similar][variaveis].reset_index()
    referencia = original.iloc[similar]['referencia'].reset_index()
    referencia = referencia.merge(original_similar, on='index', how='left')
    referencia = referencia.drop(columns=['index'])
    for ind, rw in referencia.iterrows():    
        if row['referencia'] != rw['referencia']:            
            print('--> {0}'.format(rw['referencia']))
            similares = similares.append({'referencia': row['referencia'], 'vizinho': rw['referencia']}, ignore_index = True)
similares

similares.to_feather('tavbase/knn_estado.feather')

C:\Users\duduf\AppData\Local\Temp\ipykernel_25576\2601909467.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rfm = rfm.append(
C:\Users\duduf\AppData\Local\Temp\ipykernel_25576\2601909467.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rfm = rfm.append(
C:\Users\duduf\AppData\Local\Temp\ipykernel_25576\2601909467.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rfm = rfm.append(
C:\Users\duduf\AppData\Local\Temp\ipykernel_25576\2601909467.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rfm = rfm.append(
C:\Users\duduf\AppData\Local\Temp\ipykernel_25576\2601909467.py:7: FutureWarning: The frame.append method is deprecated 

Referencia: Kentucky
Referencias Similares:
--> New Jersey
--> Wisconsin
--> Minnesota
Referencia: California
Referencias Similares:
--> New York
--> Texas
--> Washington
Referencia: Florida
Referencias Similares:
--> Illinois
--> Ohio
--> Virginia
Referencia: North Carolina
Referencias Similares:
--> Arizona
--> Colorado
--> Ohio
Referencia: Washington
Referencias Similares:
--> Pennsylvania
--> Florida
--> Michigan
Referencia: Texas
Referencias Similares:
--> Pennsylvania
--> Washington
--> Florida
Referencia: Wisconsin
Referencias Similares:
--> Minnesota
--> Massachusetts
--> New Jersey
Referencia: Utah
Referencias Similares:
--> Mississippi
--> Arkansas
--> Louisiana
Referencia: Nebraska
Referencias Similares:
--> New Hampshire
--> South Carolina
--> Vermont
Referencia: Pennsylvania
Referencias Similares:
--> Florida
--> Illinois


C:\Users\duduf\AppData\Local\Temp\ipykernel_25576\337302308.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  similares = similares.append({'referencia': row['referencia'], 'vizinho': rw['referencia']}, ignore_index = True)
C:\Users\duduf\AppData\Local\Temp\ipykernel_25576\337302308.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  similares = similares.append({'referencia': row['referencia'], 'vizinho': rw['referencia']}, ignore_index = True)
C:\Users\duduf\AppData\Local\Temp\ipykernel_25576\337302308.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  similares = similares.append({'referencia': row['referencia'], 'vizinho': rw['referencia']}, ignore_index = True)
C:\Users\duduf\AppData\Local\Temp\ipykernel_25576\33

--> Ohio
Referencia: Illinois
Referencias Similares:
--> Ohio
--> Florida
--> North Carolina
Referencia: Minnesota
Referencias Similares:
--> Delaware
--> Wisconsin
--> Massachusetts
Referencia: Michigan
Referencias Similares:
--> Virginia
--> Indiana
--> Georgia
Referencia: Delaware
Referencias Similares:
--> Minnesota
--> Massachusetts
--> Maryland
Referencia: Indiana
Referencias Similares:
--> Georgia
--> Virginia
--> Kentucky
Referencia: New York
Referencias Similares:
--> California
--> Texas
--> Washington
Referencia: Arizona
Referencias Similares:
--> Colorado
--> Tennessee
--> Massachusetts
Referencia: Virginia
Referencias Similares:
--> Michigan
--> Indiana


C:\Users\duduf\AppData\Local\Temp\ipykernel_25576\337302308.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  similares = similares.append({'referencia': row['referencia'], 'vizinho': rw['referencia']}, ignore_index = True)
C:\Users\duduf\AppData\Local\Temp\ipykernel_25576\337302308.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  similares = similares.append({'referencia': row['referencia'], 'vizinho': rw['referencia']}, ignore_index = True)
C:\Users\duduf\AppData\Local\Temp\ipykernel_25576\337302308.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  similares = similares.append({'referencia': row['referencia'], 'vizinho': rw['referencia']}, ignore_index = True)
C:\Users\duduf\AppData\Local\Temp\ipykernel_25576\33

...

C:\Users\duduf\AppData\Local\Temp\ipykernel_25576\337302308.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  similares = similares.append({'referencia': row['referencia'], 'vizinho': rw['referencia']}, ignore_index = True)
C:\Users\duduf\AppData\Local\Temp\ipykernel_25576\337302308.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  similares = similares.append({'referencia': row['referencia'], 'vizinho': rw['referencia']}, ignore_index = True)
C:\Users\duduf\AppData\Local\Temp\ipykernel_25576\337302308.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  similares = similares.append({'referencia': row['referencia'], 'vizinho': rw['referencia']}, ignore_index = True)
C:\Users\duduf\AppData\Local\Temp\ipykernel_25576\33

--> Maryland
--> Alabama
Referencia: Mississippi
Referencias Similares:
--> Utah
--> Arkansas
--> Louisiana
Referencia: Arkansas
Referencias Similares:
--> Mississippi
--> Utah
--> Connecticut
Referencia: Montana
Referencias Similares:
--> New Mexico
--> Iowa
--> Idaho
Referencia: New Hampshire
Referencias Similares:
--> Nebraska
--> South Carolina
--> Montana
Referencia: Maryland
Referencias Similares:
--> Rhode Island
--> Missouri
--> Alabama
Referencia: District of Columbia
Referencias Similares:
--> Kansas
--> Idaho
--> Wyoming
Referencia: Kansas
Referencias Similares:
--> District of Columbia
--> Idaho
--> Wyoming
Referencia: Vermont
Referencias Similares:
--> Louisiana
--> South Carolina
--> Nebraska
Referencia: Maine
Referencias Similares:
--> South Dakota


C:\Users\duduf\AppData\Local\Temp\ipykernel_25576\337302308.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  similares = similares.append({'referencia': row['referencia'], 'vizinho': rw['referencia']}, ignore_index = True)
C:\Users\duduf\AppData\Local\Temp\ipykernel_25576\337302308.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  similares = similares.append({'referencia': row['referencia'], 'vizinho': rw['referencia']}, ignore_index = True)
C:\Users\duduf\AppData\Local\Temp\ipykernel_25576\337302308.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  similares = similares.append({'referencia': row['referencia'], 'vizinho': rw['referencia']}, ignore_index = True)
C:\Users\duduf\AppData\Local\Temp\ipykernel_25576\33

--> West Virginia
--> North Dakota
Referencia: South Dakota
Referencias Similares:
--> Maine
--> West Virginia
--> North Dakota
Referencia: Idaho
Referencias Similares:
--> Iowa
--> New Mexico
--> Kansas
Referencia: North Dakota
Referencias Similares:
--> West Virginia
--> South Dakota
--> Maine
Referencia: Wyoming
Referencias Similares:
--> West Virginia
--> South Dakota
--> Maine
Referencia: West Virginia
Referencias Similares:
--> North Dakota
--> South Dakota
--> Maine


C:\Users\duduf\AppData\Local\Temp\ipykernel_25576\337302308.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  similares = similares.append({'referencia': row['referencia'], 'vizinho': rw['referencia']}, ignore_index = True)
C:\Users\duduf\AppData\Local\Temp\ipykernel_25576\337302308.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  similares = similares.append({'referencia': row['referencia'], 'vizinho': rw['referencia']}, ignore_index = True)
C:\Users\duduf\AppData\Local\Temp\ipykernel_25576\337302308.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  similares = similares.append({'referencia': row['referencia'], 'vizinho': rw['referencia']}, ignore_index = True)
C:\Users\duduf\AppData\Local\Temp\ipykernel_25576\33